# Segmented Power Law
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/thodson-usgs/ratingcurve/blob/main/docs/notebooks/segmented-power-law-demo.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/thodson-usgs/ratingcurve/main?labpath=docs%2Fnotebooks%2Fsegmented-power-law-tutorial.ipynb) 

There are several approaches to fitting a stage-discharge rating curve. 
This notebook demonstrates an simple way to approximate the classic approach, 
which uses a segmented power law.

In [ ]:
# Uncomment below to setup Google Colab. It will take a minute or so.
# %%capture
# !pip install pymc==5.1.2
# %env MKL_THREADING_LAYER=GNU
# !pip install git+https://github.com/thodson-usgs/ratingcurve.git

In [ ]:
%load_ext autoreload
%autoreload 2

import pymc as pm
import arviz as az
from ratingcurve.ratings import PowerLawRating

## Load Data
Begin by loading the Green Channel dataset

In [ ]:
from ratingcurve import data
df = data.load('green channel')
df.head()

and plotting the observations.

In [ ]:
ax = df.plot.scatter(x='q', y='stage', marker='o')
ax.set_xlabel("Discharge (cfs)")
ax.set_ylabel("Stage (ft)")

## Setup model
Now, setup the rating model.
This make take a minute the first time while the model compiles but will be faster on subsequent runs.

In [ ]:
powerrating = PowerLawRating(segments=2,
                             prior={'distribution':'uniform'})

There are of variety of ways to adjust the optimization.
Here we'll use the defaults,
which uses ADVI and runs for 200,000 iterations,
though the model should coverge well before that.

In [ ]:
trace = powerrating.fit(q=df['q'],
                        h=df['stage'], 
                        q_sigma=df['q_sigma'])

Once fit, we can plot the rating curve.

In [ ]:
powerrating.plot()

or as a table of stage-discharge values.

In [ ]:
table = powerrating.table()
table.head()

## Exercise
What happens if we choose the wrong number of segments? 
Increase the number of segments by one and rerun the model.
In fact, we can use this to select the correct number of segments,
which is demonstrated in the [model evaluation notebook](https://github.com/thodson-usgs/ratingcurve/blob/main/docs/notebooks/model-selection-tutorial.ipynb).

## Simulated Example
This example uses a simulated rating curve, which allows you to test how changing the number of segments affects the rating curve.

First, load the '3-segment simulated' tutorial dataset.

In [ ]:
sim_df = data.load('3-segment simulated')

This rating contains observations of every 0.01 inch. increment in stage, which is much more than we'd have for a natural rating.
Try sampling to `n_sample=15` or `n_sample=30` and see how that affects the model fit.

In [ ]:
# subsample the simulated rating curve
n_sample = 30
df = sim_df.sample(n_sample, random_state=12345)

ax = sim_df.plot(x='q', y='stage', color='grey', ls='-', legend=False)
df.plot.scatter(x='q', y='stage', marker='o', color='blue', ax=ax)
ax.set_xlabel("Discharge (cfs)");
ax.set_ylabel("Stage (ft)");

Setup a rating model with 3 segments

In [ ]:
powerrating = PowerLawRating(segments=3,
                             prior={'distribution':'uniform'},
                             #prior={'distribution':'normal', 'mu':[5, 8, 11], 'sigma':[1, 1, 0.2]}
                            )

now fit the model using ADVI

In [ ]:
trace = powerrating.fit(q=df['q'],
                        h=df['stage'],
                        q_sigma=None,
                        method='advi')

and visualize the results.

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
powerrating.plot(ax=ax)

# plot the original data for comparison
sim_df.plot(x='q', y='stage', color='red', ls=':', legend=False, ax=ax)

In [ ]:
powerrating.summary(var_names=["b", "a", "sigma", "hs"])

Fitting this model can be tricky.
The most common issue is a poor initialization of the breakpoints.
A fix is under development, but for now, try
1. reinitializing the model `PowerLawRating()`;
1. increasing the number of iterations for the fitting algorithm `fit(n=300_000)`;
1. a prior on the breakpoints, example, try `prior={'distribution':'normal', 'mu':[5, 9.5, 10.5], 'sigma':[1, 1, 0.2]})`, which implies we know the true breakpoint within +-0.5 ft; or
1. fitting the model with NUTS `fit(method='nuts')`.

In [ ]:
%load_ext watermark
%watermark -n -u -v -iv -w -p pytensor,xarray